In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -U sentence-transformers optimum  onnx

#!pip install onnxruntime
!pip install onnxruntime-gpu #(gpu version)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 429.3/429.3 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 93.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-c

In [ ]:
import onnxruntime as ort
print(ort.get_available_providers())

['TensorrtExecutionProvider', 'CUDAExecutionProvider', 'CPUExecutionProvider']


In [ ]:
from optimum.onnxruntime.modeling_ort import ORTModelForCustomTasks
from transformers import AutoTokenizer
#model = ORTModelForCustomTasks.from_pretrained("optimum/sbert-all-MiniLM-L6-with-pooler")
model = ORTModelForCustomTasks.from_pretrained("optimum/sbert-all-MiniLM-L6-with-pooler",provider="CUDAExecutionProvider")
tokenizer = AutoTokenizer.from_pretrained("optimum/sbert-all-MiniLM-L6-with-pooler")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/683 [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/593 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

In [ ]:
import torch
from transformers import AutoTokenizer
from optimum.onnxruntime.modeling_ort import ORTModelForCustomTasks
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
def encode_texts_onnx_batched(texts, tokenizer, model, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
        with torch.no_grad():
            outputs = model(**inputs)
            batch_vec = outputs.pooler_output.cpu().numpy()
            embeddings.append(batch_vec)
    return np.vstack(embeddings)


def build_recommender_batched(df, tokenizer, model, top_k=10, batch_size=32):
    texts = df["merged_review_text"].tolist()
    book_ids = df["book_id"].tolist()
    vectors = encode_texts_onnx_batched(texts, tokenizer, model, batch_size=batch_size)
    sim_matrix = cosine_similarity(vectors)
    recommendations = []
    for i in range(len(book_ids)):
        sims = sim_matrix[i]
        top_indices = np.argpartition(-sims, kth=top_k + 1)[:top_k + 1]
        top_indices = top_indices[top_indices != i]
        top_indices = top_indices[np.argsort(sims[top_indices])[::-1]][:top_k]

        for j in top_indices:
            recommendations.append({
                "input_book_id": book_ids[i],
                "recommended_book_id": book_ids[j],
                "similarity": sims[j]
            })

    rec_df = pd.DataFrame(recommendations)
    return rec_df, vectors, book_ids

In [ ]:
def recommend_books_by_id(input_book_id, all_embeddings, book_ids, top_k=10):
    book_id_to_idx = {bid: i for i, bid in enumerate(book_ids)}
    if input_book_id not in book_id_to_idx:
        return f"Book ID {input_book_id} not found."

    idx = book_id_to_idx[input_book_id]
    input_vector = all_embeddings[idx].reshape(1, -1)

    sims = cosine_similarity(input_vector, all_embeddings)[0]
    top_indices = sims.argsort()[::-1][1:top_k+1]

    results = []
    for i in top_indices:
        results.append({
            "input_book_id": input_book_id,
            "recommended_book_id": book_ids[i],
            "similarity": sims[i]
        })
    return pd.DataFrame(results)

In [ ]:
import pandas as pd
import gzip
import json
import os
import numpy as np
#finished
indir = '/content/drive/MyDrive/Capstone/dataset/'
review_path = os.path.join(indir, "goodreads_reviews_comics_graphic.json.gz")
save_dir = '/content/drive/MyDrive/Capstone/output_batches'
os.makedirs(save_dir, exist_ok=True)

batch_size = 5000
line_buffer = []
total_batch_seen = 0
start_batch = 71
#end_batch =108

with gzip.open(review_path, 'rt', encoding='utf-8') as f:
    for line in f:
        line_buffer.append(line)

        if len(line_buffer) == batch_size:
            total_batch_seen += 1

            if total_batch_seen < start_batch:

                line_buffer = []
                continue

            #if total_batch_seen > end_batch:
                #break

            print(f"Processing batch {total_batch_seen}...")

            book_reviews = {}
            for l in line_buffer:
                review = json.loads(l)
                book_id = review.get("book_id")
                review_text = review.get("review_text", "").strip()
                if not book_id or not review_text:
                    continue
                if book_id not in book_reviews:
                    book_reviews[book_id] = []
                book_reviews[book_id].append(review_text)

            book_texts = {
                book_id: " ".join(reviews) for book_id, reviews in book_reviews.items()
            }
            batch_df = pd.DataFrame({
                "book_id": list(book_texts.keys()),
                "merged_review_text": list(book_texts.values())
            })

            recommender_df, all_embeddings, book_ids = build_recommender_batched(batch_df, tokenizer, model)

            batch_prefix = f"batch_{total_batch_seen}"
            recommender_df.to_csv(os.path.join(save_dir, f"recommender_{batch_prefix}.csv"), index=False)
            np.save(os.path.join(save_dir, f"embeddings_{batch_prefix}.npy"), all_embeddings)
            np.save(os.path.join(save_dir, f"book_ids_{batch_prefix}.npy"), np.array(book_ids))

            print(f"Saved batch {total_batch_seen}")
            line_buffer = []


Processing batch 71...
Saved batch 71
Processing batch 72...
Saved batch 72
Processing batch 73...
Saved batch 73
Processing batch 74...
Saved batch 74
Processing batch 75...
Saved batch 75
Processing batch 76...
Saved batch 76
Processing batch 77...
Saved batch 77
Processing batch 78...
Saved batch 78
Processing batch 79...
Saved batch 79
Processing batch 80...
Saved batch 80
Processing batch 81...
Saved batch 81
Processing batch 82...
Saved batch 82
Processing batch 83...
Saved batch 83
Processing batch 84...
Saved batch 84
Processing batch 85...
Saved batch 85
Processing batch 86...
Saved batch 86
Processing batch 87...
Saved batch 87
Processing batch 88...
Saved batch 88
Processing batch 89...
Saved batch 89
Processing batch 90...
Saved batch 90
Processing batch 91...
Saved batch 91
Processing batch 92...
Saved batch 92
Processing batch 93...
Saved batch 93
Processing batch 94...
Saved batch 94
Processing batch 95...
Saved batch 95
Processing batch 96...
Saved batch 96
Processing b

In [ ]:
import numpy as np
import os
import glob

save_dir = '/content/drive/MyDrive/Capstone/output_batches'

embedding_files = sorted(glob.glob(os.path.join(save_dir, "embeddings_batch_*.npy")))
book_id_files = sorted(glob.glob(os.path.join(save_dir, "book_ids_batch_*.npy")))


all_embeddings = np.vstack([np.load(f) for f in embedding_files])


csv_files = sorted(glob.glob(os.path.join(save_dir, "recommender_batch_*.csv")))
merged_df = pd.concat([pd.read_csv(f) for f in csv_files], ignore_index=True)
merged_csv_path = os.path.join(save_dir, "recommender_all.csv")
merged_df.to_csv(merged_csv_path, index=False)


book_ids = []
for f in book_id_files:
    book_ids.extend(np.load(f, allow_pickle=True).tolist())  # save list

print("Merged embeddings shape:", all_embeddings.shape)
print("Merged book_ids length:", len(book_ids))
np.save(os.path.join(save_dir, "embeddings_all.npy"), all_embeddings)
np.save(os.path.join(save_dir, "book_ids_all.npy"), np.array(book_ids, dtype=object))

Merged embeddings shape: (405250, 384)
Merged book_ids length: 405250


In [ ]:
save_dir = '/content/drive/MyDrive/Capstone/output_batches'
from collections import defaultdict

all_embeddings = np.load(os.path.join(save_dir, "embeddings_all.npy"))
book_ids = np.load(os.path.join(save_dir, "book_ids_all.npy"), allow_pickle=True).tolist()

id_to_vecs = defaultdict(list)
for book_id, vec in zip(book_ids, all_embeddings):
    id_to_vecs[book_id].append(vec)


dedup_book_ids = []
dedup_embeddings = []

for book_id, vec_list in id_to_vecs.items():
    dedup_book_ids.append(book_id)
    avg_vec = np.mean(vec_list, axis=0)
    dedup_embeddings.append(avg_vec)


dedup_embeddings = np.vstack(dedup_embeddings)

np.save(os.path.join(save_dir, "book_ids_all_dedup.npy"), np.array(dedup_book_ids, dtype=object))
np.save(os.path.join(save_dir, "embeddings_all_dedup.npy"), dedup_embeddings)

print(f"bookidnum: {len(dedup_book_ids)}")
print(f"embedding dimension: {dedup_embeddings.shape}")

bookidnum: 89123
embedding dimension: (89123, 384)


In [ ]:
save_dir = '/content/drive/MyDrive/Capstone/output_batches'

dedup_book_ids = np.load(os.path.join(save_dir, "book_ids_all_dedup.npy"), allow_pickle=True).tolist()
dedup_embeddings = np.load(os.path.join(save_dir, "embeddings_all_dedup.npy"))

display(recommend_books_by_id("24494", dedup_embeddings, dedup_book_ids, top_k=10))

,input_book_id,recommended_book_id,similarity
0,24494,24812,0.990920
1,24494,59715,0.989255
2,24494,43070,0.989060
3,24494,24813,0.987830
4,24494,121792,0.987675
5,24494,77727,0.987423
6,24494,77730,0.987295
7,24494,24816,0.987228
8,24494,70489,0.986921
9,24494,24814,0.986716


In [ ]:
merged_csv_path = os.path.join(save_dir, "recommender_all.csv")
df = pd.read_csv(merged_csv_path)

top_k = 10
df_topk = (
    df.sort_values(by="similarity", ascending=False)
      .groupby("input_book_id")
      .head(10)
      .reset_index(drop=True)
)


df_topk_sorted = df_topk.sort_values(by="input_book_id").reset_index(drop=True)

deduped_path = os.path.join(save_dir, "recommender_all_dedup.csv")
df_topk_sorted.to_csv(deduped_path, index=False)


print(f"Deduplicated top-{top_k} recommendations saved to {deduped_path}")

Deduplicated top-10 recommendations saved to /content/drive/MyDrive/Capstone/output_batches/recommender_all_dedup.csv


In [ ]:
def recommend_all_books_by_id(all_embeddings, book_ids, top_k=10):
    all_results = []
    total = len(book_ids)

    for i, input_book_id in enumerate(book_ids):
        df = recommend_books_by_id(input_book_id, all_embeddings, book_ids, top_k=top_k)
        if isinstance(df, pd.DataFrame):
            all_results.append(df)

        if (i + 1) % 1000 == 0 or i == total - 1:
            print(f"Processed {i + 1}/{total}")

    return pd.concat(all_results, ignore_index=True)

In [ ]:
#predict by embedding

recommender_df = recommend_all_books_by_id(dedup_embeddings, dedup_book_ids, top_k=10)
recommender_df.to_csv(os.path.join(save_dir, "recommender_mean_embedding.csv"), index=False)

Processed 1000/89123
Processed 2000/89123
Processed 3000/89123
Processed 4000/89123
Processed 5000/89123
Processed 6000/89123
Processed 7000/89123
Processed 8000/89123
Processed 9000/89123
Processed 10000/89123
Processed 11000/89123


KeyboardInterrupt: 